# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

#Note that I had to pip install 'cartopy' and 'pyproj' for the maps

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,17.55,66,1,7.72,AU,1704798090
1,1,tianchang,37.9981,114.0156,-0.99,69,8,0.61,CN,1704798090
2,2,anadyr,64.7500,177.4833,-20.56,83,100,3.00,RU,1704798093
3,3,eyl,7.9803,49.8164,28.44,50,15,11.64,SO,1704798093
4,4,port-aux-francais,-49.3500,70.2167,4.54,74,100,10.29,TF,1704798027
...,...,...,...,...,...,...,...,...,...,...
557,557,timimoun,29.2639,0.2310,18.01,17,83,0.52,DZ,1704798180
558,558,callaway,30.1530,-85.5699,20.58,94,100,14.92,US,1704798347
559,559,paungde,18.4833,95.5000,26.53,36,4,2.19,MM,1704798347
560,560,coxim,-18.5067,-54.7600,27.33,73,13,2.05,BR,1704798347


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', #Obtain x and y values for scatter plot from relevant Dataframe columns (Lng & Lat)
    y='Lat',
    c='Humidity', #Set colour to be variable based on Humidity column of Dataframe
    colorbar=True, #Add colour bar for heat map readability
    size='Humidity', #Size of scatter plot points set to be variable based on Humidity column of Dataframe
    title='City Map with Humidity', #Titles and labels setting
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=['City', 'Humidity'], #Set plot to display City and Humidity values for that point when mouse hovers over
    geo=True,  #geo parameter set to True for geographic background
    tiles="OSM",  #Background source set to OpenStreetMap
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 30) &
    (city_data_df['Max Temp'] > 10) &
    (city_data_df['Humidity'] < 70) &
    (city_data_df['Wind Speed'] < 8) &
    (city_data_df['Cloudiness'] < 40)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,17.55,66,1,7.72,AU,1704798090
16,16,zahle,33.8497,35.9042,11.89,39,39,4.59,LB,1704798098
17,17,ghadamis,30.1337,9.5007,12.88,31,26,1.21,LY,1704798098
20,20,abu qurqas,27.9312,30.8384,22.07,11,29,3.99,EG,1704798099
25,25,joshimath,30.5667,79.5667,11.61,25,0,2.73,IN,1704798101
39,39,brak,27.5496,14.2714,16.10,17,0,2.40,LY,1704798107
44,44,mukalla,14.5425,49.1242,27.37,29,5,4.30,YE,1704798108
65,65,i-n-salah,27.1935,2.4607,20.16,15,3,5.49,DZ,1704798117
85,85,tura,25.5198,90.2201,18.49,51,0,1.49,IN,1704798128
87,87,isla vista,34.4133,-119.8610,11.57,56,0,0.45,US,1704798130


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy() #Copy contents of ideal_weather_df to new hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name="") #Create new column with empty strings called Hotel_Name

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name
0,0,margaret river,-33.9500,115.0667,17.55,66,1,7.72,AU,1704798090,
16,16,zahle,33.8497,35.9042,11.89,39,39,4.59,LB,1704798098,
17,17,ghadamis,30.1337,9.5007,12.88,31,26,1.21,LY,1704798098,
20,20,abu qurqas,27.9312,30.8384,22.07,11,29,3.99,EG,1704798099,
25,25,joshimath,30.5667,79.5667,11.61,25,0,2.73,IN,1704798101,
39,39,brak,27.5496,14.2714,16.10,17,0,2.40,LY,1704798107,
44,44,mukalla,14.5425,49.1242,27.37,29,5,4.30,YE,1704798108,
65,65,i-n-salah,27.1935,2.4607,20.16,15,3,5.49,DZ,1704798117,
85,85,tura,25.5198,90.2201,18.49,51,0,1.49,IN,1704798128,
87,87,isla vista,34.4133,-119.8610,11.57,56,0,0.45,US,1704798130,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = { #Note that this is the base parameter dictionary for every search
    'limit': 1,  # Limit to one result (the nearest hotel)
    'apiKey': geoapify_key #Obtain API key from api_keys
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Quality Inn Margaret River
zahle - nearest hotel: فندق الخيال الكبير
ghadamis - nearest hotel: فندق ونزريك
abu qurqas - nearest hotel: No hotel found
joshimath - nearest hotel: Snow Crest Hotel
brak - nearest hotel: فندق براك السياحي
mukalla - nearest hotel: فندق موج
i-n-salah - nearest hotel: تيدكلة
tura - nearest hotel: No hotel found
isla vista - nearest hotel: The Ritz-Carlton Bacara, Santa Barbara
el-tor - nearest hotel: Gebel El Tor
longquan - nearest hotel: 易门大酒店
murzuq - nearest hotel: فندق باريس - مرزق
iranshahr - nearest hotel: هتل قصِر
san patricio - nearest hotel: No hotel found
at taraf - nearest hotel: No hotel found
jiangyou - nearest hotel: No hotel found
mao - nearest hotel: AluaSun Far Menorca
santa elena de uairen - nearest hotel: Hotel Garibaldi
meiktila - nearest hotel: Grand Cosmos Hotel
broken hill - nearest hotel: Old Willyama Hotel
san sebastian de la gomera - nearest hotel: Hotel Jardin Tecina
luang namtha 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name,Hotel Name
0,0,margaret river,-33.9500,115.0667,17.55,66,1,7.72,AU,1704798090,,Quality Inn Margaret River
16,16,zahle,33.8497,35.9042,11.89,39,39,4.59,LB,1704798098,,فندق الخيال الكبير
17,17,ghadamis,30.1337,9.5007,12.88,31,26,1.21,LY,1704798098,,فندق ونزريك
20,20,abu qurqas,27.9312,30.8384,22.07,11,29,3.99,EG,1704798099,,No hotel found
25,25,joshimath,30.5667,79.5667,11.61,25,0,2.73,IN,1704798101,,Snow Crest Hotel
39,39,brak,27.5496,14.2714,16.10,17,0,2.40,LY,1704798107,,فندق براك السياحي
44,44,mukalla,14.5425,49.1242,27.37,29,5,4.30,YE,1704798108,,فندق موج
65,65,i-n-salah,27.1935,2.4607,20.16,15,3,5.49,DZ,1704798117,,تيدكلة
85,85,tura,25.5198,90.2201,18.49,51,0,1.49,IN,1704798128,,No hotel found
87,87,isla vista,34.4133,-119.8610,11.57,56,0,0.45,US,1704798130,,"The Ritz-Carlton Bacara, Santa Barbara"


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng', #Obtain x and y values for scatter plot from relevant Dataframe columns (Lng & Lat)
    y='Lat',
    c='Humidity', #Set colour to be variable based on Humidity column of Dataframe
    colorbar=True, #Add colour bar for heat map readability
    size='Humidity', #Size of scatter plot points set to be variable based on Humidity column of Dataframe
    title='City Map with Humidity', #Titles and labels setting
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'], #Set plot to display City, Humidity, Hotel Name and Country values for that point when mouse hovers over
    geo=True, #geo parameter set to True for geographic background
    tiles="OSM" #Background source set to OpenStreetMap
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)